In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

import cv2
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import random

import json

from glob import glob

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# GPU selection (memory)
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
# Data info.

width = 640
height = 480
channel = 3


# resize
r_w = 512
r_h = 512

total_pix = r_w * r_h * channel

In [4]:
def data_process(d_dir, r1, r2, r3):
    
    total = r1 + r2 + r3
    
    if total == 1:
        num_class = len(d_dir.keys())

        out = {}

        r_v = r2 / (r1+r2)
        
        for i in range(num_class):
            
            x_1, x_test = train_test_split(d_dir[i], test_size=r3)

            if i == 0:
                train_1 = x_1
                test_temp = x_test
            else:
                train_1.extend(x_1)
                test_temp.extend(x_test)

        random.shuffle(train_1)
        
        train_idx = int(len(train_1) * r1/(r1+r2))
        
        out['train'] = train_1[0:train_idx]
        out['valid'] = train_1[train_idx:]
        
        random.shuffle(test_temp)
        out['test'] = test_temp
    
    return out

In [5]:
image_dir1 = '/dataset_full/'
class_name1 = ['Untorn', 'Torn']

data_dir1 = {}
for i in range(len(class_name1)):
    temp_dir = image_dir1 + class_name1[i]
    img_files = [os.path.join(temp_dir, f) for f in os.listdir(temp_dir) if f.endswith('.png')]
    data_dir1[i] = img_files
print(len(data_dir1[0]), len(data_dir1[1]))

2136 2427


In [6]:
r_train = 0.8
r_valid = 0.05
r_test = 0.15
tr_data_dir = data_process(data_dir1, r_train, r_valid, r_test)

In [7]:
print(len(tr_data_dir['train']), len(tr_data_dir['valid']), len(tr_data_dir['test']))
print(len(tr_data_dir['train'])+len(tr_data_dir['valid'])+len(tr_data_dir['test']))
print(len(tr_data_dir['train'])/4563, len(tr_data_dir['valid'])/4563, len(tr_data_dir['test'])/4563)

3648 229 686
4563
0.799474030243261 0.050186280955511725 0.15033968880122725


In [8]:
def default(o):
    if isinstance(o, (np.int_, np.intc, np.intp, np.int8,
                      np.int16, np.int32, np.int64, np.uint8,
                      np.uint16,np.uint32, np.uint64)):
        return int(o)

    raise TypeError

with open('./data_split/data_dir_0801.txt', 'w') as file:
    file.write(json.dumps(tr_data_dir, default=default))